In [72]:
import importlib

import common
import util
import pbar
import ml_util

importlib.reload(common)
importlib.reload(ml_util)
importlib.reload(util)
importlib.reload(pbar)

<module 'pbar' from 'C:\\Users\\j\\IdeaProjects\\fiit_masters_thesis_project\\jupyter\\pbar.py'>

In [73]:
%matplotlib inline

import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy

from common import create_engine
from common import display_all
from common import figsize
from common import save_df, load_df
from common import save_session, load_session


from util import *

from pbar import Pbar

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots

In [3]:
SESSION_FILE_NAME = 'method-impl.session-db'

load_session(SESSION_FILE_NAME)

In [4]:
RANDOM_STATE = 123
np_random = np.random.RandomState(RANDOM_STATE)

### Load dataset

In [5]:
df = load_df('final_data.pickle')

In [6]:
df.head()

,id,url,title,perex,body,published_at,extracted_at,source_id,category,other_info,...,source_id,source_name,source_url,source_type,source_is_reliable,fb_sync_date,fb_reaction_count,fb_comment_count,fb_share_count,fb_popularity
0,355165,http://blog.healthadvocate.com/2011/08/smarter...,Smarter eating at buffets,<p>It&#8217;s easy to be fooled into thinking ...,It’s easy to be fooled into thinking that dini...,2011-08-02 17:06:00,2019-09-06 04:02:15.203668,223,"{""Health Tips"",Nutrition,""Health Information"",...","{'tags': ['Healthy Eating', 'Nutrition', 'Well...",...,223,healthadvocate.com,http://healthadvocate.com,news_website,1,2019-10-14 21:53:23.370260,0,0,0,0
1,355267,http://blog.healthadvocate.com/2013/01/soups-on/,Soup&#8217;s on!,"<p>During the colder months, eating a bowl of ...","During the colder months, eating a bowl of sou...",2013-01-15 21:31:00,2019-09-06 04:02:58.622688,223,{Nutrition},"{'tags': [], 'updated_at': '2018-03-02T11:44:17'}",...,223,healthadvocate.com,http://healthadvocate.com,news_website,1,2019-10-14 21:53:24.046867,0,0,0,0
2,355283,http://blog.healthadvocate.com/2013/04/dancing...,Dancing Your Way to Better Health,"<p>Do you enjoy dancing? If so, you may want t...","Do you enjoy dancing? If so, you may want to c...",2013-04-26 14:43:05,2019-09-06 04:03:06.256547,223,"{Wellness,Fitness}","{'tags': [], 'updated_at': '2018-03-02T11:44:16'}",...,223,healthadvocate.com,http://healthadvocate.com,news_website,1,2019-10-14 21:53:24.047008,0,0,0,0
3,355307,http://blog.healthadvocate.com/2013/07/how-eap...,How EAPs Can Benefit a Company’s Employees…and...,<p>Job stress is a leading source of stress fo...,Job stress is a leading source of stress for A...,2013-07-09 17:24:00,2019-09-06 04:03:15.869432,223,"{""Stress Management"",""Employee Assistance Prog...","{'tags': [], 'updated_at': '2018-03-02T11:43:40'}",...,223,healthadvocate.com,http://healthadvocate.com,news_website,1,2019-10-14 21:53:24.384061,0,0,0,0
4,355603,http://blog.healthadvocate.com/2015/05/healthy...,Healthy Reasons to Exercise,<p>We’re frequently exposed to public service ...,We’re frequently exposed to public service ann...,2015-05-19 11:55:40,2019-09-06 04:07:58.978488,223,"{Wellness,Fitness}","{'tags': [], 'updated_at': '2018-03-02T11:37:20'}",...,223,healthadvocate.com,http://healthadvocate.com,news_website,1,2019-10-14 21:53:26.547686,2,0,4,6


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160897 entries, 0 to 168190
Data columns (total 21 columns):
id                    160897 non-null int64
url                   160897 non-null object
title                 160897 non-null object
perex                 137532 non-null object
body                  160668 non-null object
published_at          160897 non-null datetime64[ns]
extracted_at          160897 non-null datetime64[ns]
source_id             160897 non-null int64
category              133794 non-null object
other_info            160893 non-null object
author_name           160897 non-null object
source_id             160897 non-null int64
source_name           160897 non-null object
source_url            160897 non-null object
source_type           160897 non-null object
source_is_reliable    160897 non-null int64
fb_sync_date          160897 non-null datetime64[ns]
fb_reaction_count     160897 non-null int64
fb_comment_count      160897 non-null int64
fb_share_count  

-----

Rozdelenie hodnot popularity do 5 skupin

- `0 - 0.5`
- `0.5 - 0.75`
- `0.75 - 0.9`
- `0.9 - 0.95`
- `0.95 - 1`

In [8]:
def add_labels(df, quantiles, column='fb_popularity'):
    df = df.copy()
    label_str = f'{column}_label'
    
    df[label_str] = -1
    
    label = 1    
    for i in range(len(quantiles) - 1):
        low = df[column].quantile(quantiles[i])
        high = df[column].quantile(quantiles[i + 1])
        
        df.loc[(low <= df[column]) & (df[column] <= high), label_str] = label
        
        label += 1
    
    return df.copy()

In [9]:
quantiles = [
    0,
    .50,
    .75,
    .90,
    .95,
    1
]

cols = [
    'fb_reaction_count',
    'fb_comment_count',
    'fb_share_count',
    'fb_popularity'    
]

for i in cols:
    print(df[i].quantile(quantiles))
    df = add_labels(df, quantiles, column=i)

0.00          0.0
0.50          0.0
0.75         50.0
0.90        436.0
0.95       1301.0
1.00    3929532.0
Name: fb_reaction_count, dtype: float64
0.00         0.0
0.50         0.0
0.75         7.0
0.90        85.0
0.95       290.0
1.00    714968.0
Name: fb_comment_count, dtype: float64
0.00         0.0
0.50         1.0
0.75        33.0
0.90       185.0
0.95       503.0
1.00    572708.0
Name: fb_share_count, dtype: float64
0.00          0.0
0.50          1.0
0.75         99.0
0.90        734.0
0.95       2126.0
1.00    5197586.0
Name: fb_popularity, dtype: float64


Pri jednotlivych zlozkach sme pri tomto rozdeleni nasli len 4 skupiny (lebo 1 == 2)

---

Jednoducha heuristika: ak je zdroj nedoveryhodny tak aj clanok je nedoveryhodny

In [10]:
df['is_fake_news_label'] = df.source_is_reliable.replace({0:1, 1:0})

---

### data cleanup

In [11]:
# clear body, perex, etc from html....

In [12]:
# drop rows without body
df = df[~df.body.isnull()]
df = df[~df.title.isnull()]

## Feature engineering

dve hlavne skupinky dat

- metadata (rozne)
- obsahove

In [13]:
label_names = list(filter(lambda x: x.endswith('_label'), df.columns))

In [14]:
f_df = pd.DataFrame()

In [15]:
# labely
f_df = pd.concat([f_df] + [df[label_name] for label_name in label_names], axis=1)

df.info()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob

In [17]:
def title_basic_features(df):
    content_cv = CountVectorizer()
    data = content_cv.fit_transform(df.body)

    res = pd.DataFrame(index=df.index)
    
    res['title_word_count'] = data.sum(axis=1)
    res['title_char_length'] = df.title.apply(lambda x: len(x))
    
    return res

def title_sentiment(df):
    return empty_features(df)

In [18]:
def content_basic_features(df):
    content_cv = CountVectorizer()
    # todo preprocess body
    data = content_cv.fit_transform(df.body)

    res = pd.DataFrame(index=df.index)
    
    res['content_word_count'] = data.sum(axis=1)
    res['content_char_length'] = df.body.apply(lambda x: len(x))
    
    return res

def content_sentiment(df):
    return empty_features(df)

In [19]:
features = [
    title_basic_features,
    title_sentiment,
    
    content_basic_features,
    content_sentiment
]

In [20]:
def add_features(df, f_df):
    f_df = f_df.copy()
    for feature_generator in Pbar(features, refresh_rate=1, action_names=[i.__name__ for i in features]):
        f_df = pd.concat([f_df, feature_generator(df)], axis=1)
        
    return f_df

In [21]:
final_df = add_features(df, f_df)

[======================================            ] (processing: content_sentiment) -- 3 / 4 -- (finished)
[==================================================] -- 4 / 4

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160668 entries, 0 to 168190
Data columns (total 9 columns):
fb_reaction_count_label    160668 non-null int64
fb_comment_count_label     160668 non-null int64
fb_share_count_label       160668 non-null int64
fb_popularity_label        160668 non-null int64
is_fake_news_label         160668 non-null int64
title_word_count           160668 non-null int64
title_char_length          160668 non-null int64
content_word_count         160668 non-null int64
content_char_length        160668 non-null int64
dtypes: int64(9)
memory usage: 12.3+ MB


---

## Rozdelenie dat

In [23]:
train, test, validation = tuple(split_data(final_df, sizes=[2, 2, 1], shuffle=True, np_random=np_random))

In [24]:
print([len(i) for i in [train,test,validation]])

[64268, 64267, 32133]


In [25]:
label_names

['fb_reaction_count_label',
 'fb_comment_count_label',
 'fb_share_count_label',
 'fb_popularity_label',
 'is_fake_news_label']

In [26]:
data = split_X_y_all(train, test, validation, selected_label='fb_popularity_label', all_labels=label_names)

In [34]:
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier

In [40]:
def fit_predict(clf, data):
    clf.fit(data.train.X, data.train.y)
    return clf.predict(data.test.X)

In [74]:
classifiers = [
    RandomForestClassifier(n_estimators=100),

]

pbar_conf = {'length': len(classifiers), 'pbar_width': 52, 'action_names': [i.__class__.__name__ for i in classifiers]}

predictions = list(Pbar((fit_predict(clf, data) for clf in classifiers), **pbar_conf))

for p in predictions:
    print(classification_report(data.test.y, p))
    print('-' * 54)


[====================================================] -- 1 / 1 -- (finished) stClassifier) -- 0 / 1
------------------------------------------------------
              precision    recall  f1-score   support

           1       0.54      0.70      0.61     29336
           2       0.36      0.33      0.35     18940
           3       0.25      0.17      0.20      9597
           4       0.10      0.03      0.05      3206
           5       0.11      0.04      0.06      3188

    accuracy                           0.45     64267
   macro avg       0.27      0.25      0.25     64267
weighted avg       0.40      0.45      0.42     64267

------------------------------------------------------


In [32]:
save_session(SESSION_FILE_NAME)